## Notebook Purposes
This notebook is used to convert the existing dataset images to numpy arrays, pairing them up with other images to increase input complexity.

In [24]:
# Create new CSV files which only include examples that contain cloud filtered images along with composite images

import pandas as pd
import glob

#read through all files in training, val, and test sets
train = pd.read_csv('../input/forestnet/ForestNetDataset/train.csv')
val = pd.read_csv('../input/forestnet/ForestNetDataset/val.csv')
test = pd.read_csv('../input/forestnet/ForestNetDataset/test.csv')

train_dict = {'label':[], 'merged_label': [], 'latitude': [], 'longitude': [], 'year': [], 'example_path': []}
val_dict = {'label':[], 'merged_label': [], 'latitude': [], 'longitude': [], 'year': [], 'example_path': []}
test_dict = {'label':[], 'merged_label': [], 'latitude': [], 'longitude': [], 'year': [], 'example_path': []}


#check to see if cloud filtered image is available
for example_set, set_dict, set_path in zip([train, val, test], [train_dict, val_dict, test_dict], ['train_new.csv', 'val_new.csv', 'test_new.csv']):   
    for i in range(example_set.shape[0]):
            img_path = f'../input/forestnet/ForestNetDataset/{example_set["example_path"][i]}/images/visible'
            list_of_files = glob.glob(img_path + '/*.png')
            if len(list_of_files) > 1:
                  for col in ['label', 'merged_label', 'latitude', 'longitude', 'year', 'example_path']:
                    set_dict[col].append(example_set[col][i])
            else:
              continue
    set_df = pd.DataFrame(set_dict)
    set_df.to_csv('../input/forestnet/ForestNetDataset/' + set_path, index=False)

            


In [31]:
# SOLUTION TO ONLY INCLUDE EXAMPLES WITH COMPOSITE AND CLOUD FILTERED IMAGES
from PIL import Image
from numpy import asarray, concatenate, save
import pandas as pd
import glob

#path to set csv files
train = pd.read_csv('../input/forestnet/ForestNetDataset/train_new.csv')
val = pd.read_csv('../input/forestnet/ForestNetDataset/val_new.csv')
test = pd.read_csv('../input/forestnet/ForestNetDataset/test_new.csv')

# path_to_images = "../input/forestnet/ForestNetDataset/"f'{x[1]}'"/images/visible/composite.png"
path_to_images = "../input/forestnet/ForestNetDataset/examples/0.353895870891265_102.2666626652374/images"
additional_path1 = "/visible"

img_path = path_to_images + additional_path1

list_of_files = glob.glob(img_path + "/*.png")

if len(list_of_files) > 1:
    #loads example's composite image
    composite_img = Image.open(list_of_files[-1])
    #loads most recent cloud filtered image
    cloudf_img = Image.open(list_of_files[-2])
    #convert to np-array
    np_composite_img = asarray(composite_img)
    np_cloudf_img = asarray(cloudf_img)
    # COULD ALSO USE DIFFERENT AXIS TO GET NON-SQUARE IMAGE, I DON'T THINK THAT'S A GOOD IDEA
    np_new = concatenate((np_composite_img, np_cloudf_img), 2)
    save(img_path + "/concatenated_images.npy", np_new)
    


    